<h1>To mount drive</h1>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# to change the working directory
%cd /content/drive/MyDrive/Colab Notebooks/Steganography

/content/drive/.shortcut-targets-by-id/1oaZCwXh2SHDAQ3FtWzvS1pKkn3EYFRo1/Colab Notebooks/Steganography


<h1>The Imports</h1>

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
import joblib
import tempfile
import cv2
import subprocess


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tempfile.tempdir = "/tmp"

<h1>To Extract Audio From Video</h1>

In [4]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=66208b2d9b6d1e989576635238e740a7097925da7d5978b8758d3cc40a57fc48
  Stored in directory: /root/.cache/pip/wheels/30/33/46/5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg


In [5]:
# To extract audio from video 

def convert_video_to_audio(video_file, output_ext="wav"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)






In [6]:
vf = 'Video and Audio files/covervid.mp4'
convert_video_to_audio(vf)

<h1>To make temporary file of list of frames

In [7]:
# Creating temporary file and deleting it (experiment)

temp = tempfile.TemporaryFile()

try:
    a = np.array([[1, 2, 3],[4, 5, 6]])
    b = np.array([[45,67 , 63],[14, 25, 36]])
    print("Original array:")
    print(a)
    print(a.shape)
    a_bytes = a.tobytes()
    b_bytes = b.tobytes()
    print(len(a_bytes))
    print(len(b_bytes))
    temp.write(a_bytes)
    temp.write(b_bytes)
    temp.seek(0)
    
    
    print("After loading, content of the temp file:")
    for i in range(2):
      read_bytes = temp.read(len(a_bytes))
      print("in the temp file: ", read_bytes)
      a2 = np.frombuffer(read_bytes, dtype=a.dtype)
      a2 = a2.reshape(a.shape)
      print(a2)
    #print(temp.read())
   
finally:
    temp.close()

Original array:
[[1 2 3]
 [4 5 6]]
(2, 3)
48
48
After loading, content of the temp file:
in the temp file:  b'\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00'
[[1 2 3]
 [4 5 6]]
in the temp file:  b'-\x00\x00\x00\x00\x00\x00\x00C\x00\x00\x00\x00\x00\x00\x00?\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x19\x00\x00\x00\x00\x00\x00\x00$\x00\x00\x00\x00\x00\x00\x00'
[[45 67 63]
 [14 25 36]]


In [8]:
cover_vid = 'Video and Audio files/covervid.mp4'
secret_vid = 'Video and Audio files/secretvid.mp4'

In [21]:
# cover video to frames in a temporary file:
cap1 = cv2.VideoCapture(cover_vid)
fps1 = cap1.get(cv2.CAP_PROP_FPS)
temp_cover = tempfile.TemporaryFile()
count1 = 0
while True:
      is_read, frame = cap1.read()
      if not is_read:
          # break out of the loop if there are no frames to read
          break
      if count1 == 0:
        cover_frame = frame
      temp_cover.write(frame.tobytes())
      count1 += 1
temp_cover.seek(0)
print('total number of frames in cover =', count1)


# secret video to frames in a temporary file:

cap2 = cv2.VideoCapture(secret_vid)
fps2 = cap2.get(cv2.CAP_PROP_FPS)
temp_secret = tempfile.TemporaryFile()
count2 = 0
while True:
      is_read, frame = cap2.read()
      if not is_read:
          # break out of the loop if there are no frames to read
          break
      if count2 == 0:
        secret_frame = frame
      temp_secret.write(frame.tobytes())
      count2 += 1
temp_secret.seek(0)
print('total number of frames in secret =', count2)


total number of frames in cover = 2403
total number of frames in secret = 893


In [22]:
# checking if the above code did the right job

cover_frame_bytes = cover_frame.tobytes()
secret_frame_bytes = secret_frame.tobytes()
read_cover_bytes = temp_cover.read(len(cover_frame_bytes))
frame_obt = np.frombuffer(read_cover_bytes, dtype = cover_frame.dtype)
frame_obt = frame_obt.reshape(cover_frame.shape)
print("obtained frame :" , frame_obt[:5])
print("the original and the obtained are same : " , np.array_equal(frame_obt,cover_frame))

obtained frame : [[[238 214 156]
  [238 214 156]
  [238 214 156]
  ...
  [232 211 194]
  [231 210 193]
  [231 210 193]]

 [[238 214 156]
  [238 214 156]
  [238 214 156]
  ...
  [231 210 193]
  [231 210 193]
  [231 210 193]]

 [[238 214 156]
  [238 214 156]
  [238 214 156]
  ...
  [231 210 193]
  [231 210 193]
  [231 210 193]]

 [[238 214 156]
  [238 214 156]
  [238 214 156]
  ...
  [231 210 193]
  [231 210 193]
  [231 210 193]]

 [[238 214 156]
  [238 214 156]
  [238 214 156]
  ...
  [231 210 193]
  [231 210 193]
  [231 210 193]]]
the original and the obtained are same :  True


In [23]:
# close the temp file, which will also delete it

temp_cover.close()
temp_secret.close()

<h1>To push this code to Github</h1>

In [31]:
!git config --global user.email "manohardahal40@gmail.com"
!git config --global user.name "manohar322028"

%cd /content/drive/MyDrive
!git init 'Colab Notebooks'

%cd /content/drive/MyDrive/Colab Notebooks/Steganography

!git add NOTEBOOKS/Video_Steganography.ipynb

!git commit -m "creation of temporary files"

!git push origin master



/content/drive/MyDrive
Reinitialized existing Git repository in /content/drive/.shortcut-targets-by-id/1oaZCwXh2SHDAQ3FtWzvS1pKkn3EYFRo1/Colab Notebooks/.git/
/content/drive/.shortcut-targets-by-id/1oaZCwXh2SHDAQ3FtWzvS1pKkn3EYFRo1/Colab Notebooks/Steganography
Auto packing the repository in background for optimum performance.
See "git help gc" for manual housekeeping.
[master 60b8eda] creation of temporary files
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Steganography/NOTEBOOKS/Video_Steganography.ipynb (83%)
fatal: could not read Username for 'https://github.com': No such device or address


<h1>For Model</h1>

In [3]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/'
dataset_path = os.path.join(root_dir,'Steganography/Dataset')

train_csv = os.path.join(root_dir,'train_dataset.csv')
valid_csv = os.path.join(root_dir,'validation_dataset.csv')
test_csv = os.path.join(dataset_path,'validation_dataset.csv')
train_folder = os.path.join(dataset_path,'train')
validation_folder = os.path.join(dataset_path,'valid')

In [ ]:
cover = '/content/drive/MyDrive/Colab Notebooks/Steganography/Lion.jpg'
secret = '/content/drive/MyDrive/Colab Notebooks/Steganography/Tiger.jpg'

dataset = [{
        'cover_image':cover,
        'secret_image':secret
    }]
dataframe = pd.DataFrame(dataset)
cover = Image.open(cover)
secret = Image.open(secret)


In [ ]:
video_file2 = os.path.join(root_dir,'video_file.mp4')

In [ ]:
# framelist2, framecount2 = toFrames(video_file2)
toFrames(video_file2)

total number of frames = 893


In [ ]:
cap1 = cv2.VideoCapture(video_file1)
fps1 = cap1.get(cv2.CAP_PROP_FPS)

cap2 = cv2.VideoCapture(video_file2)
fps2 = cap2.get(cv2.CAP_PROP_FPS)

del cap1
del cap2

In [ ]:
# frames = [os.path.join(root_dir,'video_file-opencv',item) for item in dirs]
# frames[:5]

In [ ]:
dirs1=os.listdir(os.path.join(root_dir,'covervid-opencv'))
print(len(dirs1))

2402


In [ ]:
frames1 = [os.path.join(root_dir,'video_file-opencv',item) for item in dirs1]

In [ ]:
dirs2=os.listdir(os.path.join(root_dir,'video_file-opencv'))
print(len(dirs2))

893


In [ ]:
frames2 = [os.path.join(root_dir,'video_file-opencv',item) for item in dirs2]

In [ ]:
len(frames2)

893

In [ ]:
cover = Image.open(frames1[0])

In [ ]:
cover.size

(600, 600)

In [ ]:
%matplotlib inline

In [ ]:

IMG_SIZE = 64
LEARNING_RATE  = 0.001
COVER_LOSS_WEIGHT = 1
SECRET_LOSS_WEIGHT = 1
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 1
EPOCHS = 1000
DECODER_LOSS_WEIGHT = 1

In [ ]:
cover.size

(600, 600)

In [ ]:
# to reverse
cover_size = tuple(reversed(cover.size))

In [ ]:
imagetransformation = {
    'train_transforms':torchvision.transforms.Compose([torchvision.transforms.Resize((IMG_SIZE,IMG_SIZE)),torchvision.transforms.ToTensor()]),
    'valid_transforms':torchvision.transforms.Compose([torchvision.transforms.Resize((IMG_SIZE,IMG_SIZE)),torchvision.transforms.ToTensor()]),
    'test_transforms':torchvision.transforms.Compose([torchvision.transforms.Resize(cover.size),torchvision.transforms.ToTensor()])
}
#conversion of image into resized format and return of trnasformed image
class SteganoDataset(torch.utils.data.Dataset):
    def __init__(self,dataset_csv,transforms,type_of_dataset,size='complete'):
        #self.dataset = pd.read_csv(dataset_csv)
        self.dataset = dataset_csv
        self.dataset = self.dataset.reset_index(drop=True)
        if size !='complete':
            self.dataset = self.dataset[:2]
        self.transforms = transforms
        self.type = type_of_dataset
    
    def __getitem__(self,index):
        cover_image = self.dataset.iloc[index]['cover_image']
        secret_image = self.dataset.iloc[index]['secret_image']
        # print(cover_image)
        #cover_image = Image.open(os.path.join('/content/drive/MyDrive/Colab Notebooks','Valid',cover_image))
        #secret_image = Image.open(os.path.join('/content/drive/MyDrive/Colab Notebooks','Valid',secret_image))
        transformed_cover_image = self.transforms(cover)
        transformed_secret_image = self.transforms(secret)

        if self.type == 'train':
            return {
                'cover_image':transformed_cover_image,
                'secret_image':transformed_secret_image,
            }
        else:
            return {
                'cover_image':transformed_cover_image,
                'secret_image':transformed_secret_image,
            }
    
    def __len__(self):
      return len(self.dataset)
class PrepNetwork1(nn.Module):
    def __init__(self):
        super(PrepNetwork1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=3,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv4 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv6 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
    
    def forward(self,secret_image):
        output_1 = F.relu(self.conv1(secret_image))
        output_2 = F.relu(self.conv2(secret_image))
        output_3 = F.relu(self.conv3(secret_image))
        
        concatenated_image = torch.cat([output_1,output_2,output_3],dim=1)
        output_4 = F.relu(self.conv4(concatenated_image))
        output_5 = F.relu(self.conv5(concatenated_image))
        output_6 = F.relu(self.conv6(concatenated_image))
        
        final_concat_image = torch.cat([output_4,output_5,output_6],dim=1)
        return final_concat_image

class HidingNetwork(nn.Module):
    def __init__(self):
        super(HidingNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=68,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=68,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=68,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv4 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv6 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv7 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv8 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv9 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv10 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv11 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv12 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv13 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv14 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv15 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.final_layer = nn.Conv2d(in_channels=65,out_channels=3,kernel_size=(3,3),stride=1,padding=1)
        
    def forward(self,secret_image_1,cover_image):
        concatenated_secrets = torch.cat([cover_image,secret_image_1],dim=1)
        
        output_1 = F.relu(self.conv1(concatenated_secrets))
        output_2 = F.relu(self.conv2(concatenated_secrets))
        output_3 = F.relu(self.conv3(concatenated_secrets))
        concat_1 = torch.cat([output_1,output_2,output_3],dim=1)
        
        output_4 = F.relu(self.conv4(concat_1))
        output_5 = F.relu(self.conv5(concat_1))
        output_6 = F.relu(self.conv6(concat_1))
        concat_2 = torch.cat([output_4,output_5,output_6],dim=1)
        
        output_7 = F.relu(self.conv7(concat_2))
        output_8 = F.relu(self.conv8(concat_2))
        output_9 = F.relu(self.conv9(concat_2))
        concat_3 = torch.cat([output_7,output_8,output_9],dim=1)
        
        output_10 = F.relu(self.conv10(concat_3))
        output_11 = F.relu(self.conv11(concat_3))
        output_12 = F.relu(self.conv12(concat_3))
        concat_4 = torch.cat([output_10,output_11,output_12],dim=1)
        
        output_13 = F.relu(self.conv13(concat_4))
        output_14 = F.relu(self.conv14(concat_4))
        output_15 = F.relu(self.conv15(concat_4))
        concat_5 = torch.cat([output_13,output_14,output_15],dim=1)
        
        output_converted_image = F.relu(self.final_layer(concat_5))
        
        return output_converted_image
class Encoder(nn.Module):
    def __init__(self,prep_network_1,hiding_network):
        super(Encoder, self).__init__()
        self.prep_network1 = prep_network_1
        self.hiding_network = hiding_network
    
    def forward(self,cover_image,secret_image_1):
        encoded_secret_image_1 = self.prep_network1(secret_image_1)
        
        hidden_image = self.hiding_network(encoded_secret_image_1,
                                           cover_image
                                          )
#         hidden_image = (0.01**0.5)*torch.randn(hidden_image.size(),device=device)
        return hidden_image
class RevealNetwork1(nn.Module):
    def __init__(self):
        super(RevealNetwork1,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=3,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv4 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv6 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv7 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv8 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv9 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv10 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv11 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv12 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv13 = nn.Conv2d(in_channels=65,out_channels=50,kernel_size=(3,3),stride=1,padding=1)
        self.conv14 = nn.Conv2d(in_channels=65,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv15 = nn.Conv2d(in_channels=65,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.final_layer = nn.Conv2d(in_channels=65,out_channels=3,kernel_size=(3,3),stride=1,padding=1)    
    
    def forward(self,hidden_image):
        
        output_1 = F.relu(self.conv1(hidden_image))
        output_2 = F.relu(self.conv2(hidden_image))
        output_3 = F.relu(self.conv3(hidden_image))
        concat_1 = torch.cat([output_1,output_2,output_3],dim=1)
        
        output_4 = F.relu(self.conv4(concat_1))
        output_5 = F.relu(self.conv5(concat_1))
        output_6 = F.relu(self.conv6(concat_1))
        concat_2 = torch.cat([output_4,output_5,output_6],dim=1)
        
        output_7 = F.relu(self.conv7(concat_2))
        output_8 = F.relu(self.conv8(concat_2))
        output_9 = F.relu(self.conv9(concat_2))
        concat_3 = torch.cat([output_7,output_8,output_9],dim=1)
        
        output_10 = F.relu(self.conv10(concat_3))
        output_11 = F.relu(self.conv11(concat_3))
        output_12 = F.relu(self.conv12(concat_3))
        concat_4 = torch.cat([output_10,output_11,output_12],dim=1)
        
        output_13 = F.relu(self.conv13(concat_4))
        output_14 = F.relu(self.conv14(concat_4))
        output_15 = F.relu(self.conv15(concat_4))
        concat_5 = torch.cat([output_13,output_14,output_15],dim=1)
        
        output_revealed_image = F.relu(self.final_layer(concat_5))
        
        return output_revealed_image
class Decoder(nn.Module):
    def __init__(self,reveal_network_1):
        super(Decoder, self).__init__()
        self.reveal_network_1 = reveal_network_1
    
    def forward(self,hidden_image):
        reveal_image_1 = self.reveal_network_1(hidden_image)
        return reveal_image_1
class SteganoModel(nn.Module):
    def __init__(self,encoder,decoder):
        super(SteganoModel,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,cover_image,secret_image_1,hidden_image,mode):
        if mode == 'full':
            for param in self.encoder.parameters():
                param.requires_grad = True
            for param in self.decoder.parameters():
                param.requires_grad = False
            hidden_image = self.encoder(cover_image,secret_image_1)
            reveal_image_1 = self.decoder(hidden_image)
            return hidden_image,reveal_image_1
        elif mode == 'encoder':
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = False
            hidden_image = self.encoder(cover_image,secret_image_1)
            return hidden_image
        elif mode == 'decoder':
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = True
            
            reveal_image1 = self.decoder(hidden_image)
            return reveal_image1

filename = os.path.join("/content/drive/MyDrive/Colab Notebooks/Steganography/Completed_model.joblib")
loaded_model = joblib.load(filename)
#joblib.dump(model, filename)
prep_1 = PrepNetwork1()
hiding_network = HidingNetwork()

encoder = Encoder(prep_1,hiding_network)

reveal_1 = RevealNetwork1()


decoder = Decoder(reveal_1)

model = SteganoModel(encoder,decoder)
model.to(device)
class SteganoLoss(nn.Module):
    def __init__(self,cover_weight,secret_weight):
        super(SteganoLoss,self).__init__()
        self.cover_weight = cover_weight
        self.secret_weight = secret_weight
        
    def forward(self,predicted_cover_image,cover_image,
               predicted_secret_image_1,secret_image_1):
    
        cover_loss = self.cover_weight*(F.mse_loss(predicted_cover_image,cover_image))
        secret_loss = self.secret_weight*(F.mse_loss(predicted_secret_image_1,secret_image_1)) 
        return cover_loss + secret_loss

class DecoderLoss(nn.Module):
    def __init__(self,decoder_loss_weight):
        super(DecoderLoss,self).__init__()
        self.decoder_loss_weight = decoder_loss_weight
    
    def forward(self,reveal_output1,secret_image_1):
        reveal1 = self.decoder_loss_weight*F.mse_loss(reveal_output1,secret_image_1)
        return reveal1
'''
#training_csv_path = os.path.join(dataset_path,train_csv)
#validation_csv_path = os.path.join(dataset_path,valid_csv)
test_csv_path = os.path.join('/content/drive/MyDrive/Colab Notebooks','test_dataset.csv')

#training_dataset = SteganoDataset(training_csv_path,imagetransformation['train_transforms'],'train','complete')
#valid_dataset = SteganoDataset(validation_csv_path,imagetransformation['valid_transforms'],'valid')
test_dataset = SteganoDataset(test_csv_path,imagetransformation['test_transforms'],'Valid')
print(test_dataset)
'''
test_dataset = SteganoDataset(dataframe,imagetransformation['test_transforms'],'Valid')

'''
train_data_loader = torch.utils.data.DataLoader(training_dataset, 
                                                batch_size = TRAIN_BATCH_SIZE, 
                                                shuffle=True,
                                               drop_last = True,
                                               num_workers = 0
                                               )
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, 
                                                batch_size = VALID_BATCH_SIZE, 
                                                shuffle=True,
                                                drop_last = True,
                                                num_workers = 0
                                               )
                                               '''
test_data_loader = torch.utils.data.DataLoader(test_dataset, 
                                                batch_size = VALID_BATCH_SIZE, 
                                                shuffle=True,
                                                drop_last = True,
                                                num_workers = 0
                                               )



In [ ]:
'''
full_model_criterion = SteganoLoss(SECRET_LOSS_WEIGHT,COVER_LOSS_WEIGHT)

full_model_optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

decoder_criterion = DecoderLoss(DECODER_LOSS_WEIGHT)
'''

'\nfull_model_criterion = SteganoLoss(SECRET_LOSS_WEIGHT,COVER_LOSS_WEIGHT)\n\nfull_model_optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)\n\ndecoder_criterion = DecoderLoss(DECODER_LOSS_WEIGHT)\n'

In [ ]:
'''
def train(model,epochs,decoder_criterion,full_model_optimizer,full_model_criterion,learning_rate,training_iterator,valid_iterator,print_every=50):
    
    training_full_model_loss_list = []
    decoder_loss_list = []
    valid_loss_list = []
    
    for epoch in range(epochs):
        for index,training_dict in enumerate(training_iterator):

            cover_image = training_dict['cover_image']
            cover_image = cover_image.to(device)

            secret_image_1 = training_dict['secret_image']
            secret_image_1 = secret_image_1.to(device)

            
            full_model_optimizer.zero_grad()

            encoder_output = model(cover_image,secret_image_1,secret_image_1,'encoder')

            hidden_image,reveal_image_1 = model(cover_image,secret_image_1,secret_image_1,
                                                                'full')

            full_model_loss = full_model_criterion(hidden_image,cover_image,
                             reveal_image_1,secret_image_1
                            )
            full_model_loss.backward()
            full_model_optimizer.step()

            full_model_optimizer.zero_grad()
            reveal_output1 = model(cover_image,secret_image_1,
                                                                 encoder_output,'decoder')
            decoder_loss = decoder_criterion(reveal_output1,secret_image_1)

            decoder_loss.backward()
            full_model_optimizer.step()

        training_full_model_loss_list.append(full_model_loss)
        decoder_loss_list.append(decoder_loss)
        if epoch % print_every == 0:
            print("Training full model loss at {} epochs is: {}".format(epoch, full_model_loss))
            print("Training decoder loss at {} epochs is: {}".format(epoch, decoder_loss))

    return model, training_full_model_loss_list,decoder_loss_list
'''

'\ndef train(model,epochs,decoder_criterion,full_model_optimizer,full_model_criterion,learning_rate,training_iterator,valid_iterator,print_every=50):\n    \n    training_full_model_loss_list = []\n    decoder_loss_list = []\n    valid_loss_list = []\n    \n    for epoch in range(epochs):\n        for index,training_dict in enumerate(training_iterator):\n\n            cover_image = training_dict[\'cover_image\']\n            cover_image = cover_image.to(device)\n\n            secret_image_1 = training_dict[\'secret_image\']\n            secret_image_1 = secret_image_1.to(device)\n\n            \n            full_model_optimizer.zero_grad()\n\n            encoder_output = model(cover_image,secret_image_1,secret_image_1,\'encoder\')\n\n            hidden_image,reveal_image_1 = model(cover_image,secret_image_1,secret_image_1,\n                                                                \'full\')\n\n            full_model_loss = full_model_criterion(hidden_image,cover_image,\n          

Training full model loss at 0 epochs is: 0.14862200617790222
Training decoder loss at 0 epochs is: 0.056159209460020065
Training full model loss at 50 epochs is: 0.010316253639757633
Training decoder loss at 50 epochs is: 0.006905762478709221
Training full model loss at 100 epochs is: 0.005915718153119087
Training decoder loss at 100 epochs is: 0.004135390743613243
Training full model loss at 150 epochs is: 0.00597141869366169
Training decoder loss at 150 epochs is: 0.003256219904869795
Training full model loss at 200 epochs is: 0.004562869668006897
Training decoder loss at 200 epochs is: 0.0032390698324888945
Training full model loss at 250 epochs is: 0.004033954814076424
Training decoder loss at 250 epochs is: 0.00324784847907722
Training full model loss at 300 epochs is: 0.0028372027445584536
Training decoder loss at 300 epochs is: 0.002142705488950014
Training full model loss at 350 epochs is: 0.003511040238663554
Training decoder loss at 350 epochs is: 0.0028315880335867405
Training full model loss at 400 epochs is: 0.006889783777296543
Training decoder loss at 400 epochs is: 0.006818610243499279
Training full model loss at 450 epochs is: 0.0030810432508587837
Training decoder loss at 450 epochs is: 0.0027422530110925436
Training full model loss at 500 epochs is: 0.0026186055038124323
Training decoder loss at 500 epochs is: 0.00222468632273376
Training full model loss at 550 epochs is: 0.0022935508750379086
Training decoder loss at 550 epochs is: 0.0016986110713332891
Training full model loss at 600 epochs is: 0.0017199815483763814
Training decoder loss at 600 epochs is: 0.0012137810699641705
Training full model loss at 650 epochs is: 0.002981864381581545
Training decoder loss at 650 epochs is: 0.002139737131074071
Training full model loss at 700 epochs is: 0.00157183688133955
Training decoder loss at 700 epochs is: 0.0011825517285615206
Training full model loss at 750 epochs is: 0.001144192647188902
Training decoder loss at 750 epochs is: 0.0006921886233612895
Training full model loss at 800 epochs is: 0.0016071755671873689
Training decoder loss at 800 epochs is: 0.0011148437624797225
Training full model loss at 850 epochs is: 0.0016389719676226377
Training decoder loss at 850 epochs is: 0.0010738647542893887
Training full model loss at 900 epochs is: 0.0011910195462405682
Training decoder loss at 900 epochs is: 0.0007727841730229557
Training full model loss at 950 epochs is: 0.0011191064259037375
Training decoder loss at 950 epochs is: 0.0007212755153886974

In [ ]:
#model, training_full_model_loss_list,decoder_loss_list = train(model,EPOCHS,decoder_criterion,full_model_optimizer,full_model_criterion,LEARNING_RATE,train_data_loader,valid_data_loader)

In [ ]:

def predict(model,iterator,mode):

    predict_dict = next(iter(iterator))
    cover_image = predict_dict['cover_image']
    # print(cover_image)
    cover_image = cover_image.to(device)
    secret_image_1 = predict_dict['secret_image']
    secret_image_1 = secret_image_1.to(device)

    model.eval()

    if mode =='test':
      arnold = Arnold(6, 40, 33)
      coeffs, image = wtransform(cover_image[0].permute(1,2,0))
      soeffs, simage = wtransform(secret_image_1[0].permute(1,2,0))
      temp = [[],[],[]]
      coeff, cA, cH, cV, cD, soeff, sA, sH, sV, sD = [temp[:] for i in range(10)]
      
      # cA = arnold.applyTransformTo(cA)
      # cA = arnold.applyInverseTransformTo(cA)
      for ch in range(3):
        [(cA[ch], (cH[ch], cV[ch], cD[ch]))] = coeffs[ch]
        [(sA[ch], (sH[ch], sV[ch], sD[ch]))] = soeffs[ch]
        # coeff[ch]= [(cA, (cH, cV, cD))]
      
      cAA = torch.from_numpy(np.stack((cA[0], cA[1], cA[2]), axis=-1)).reshape(*cover_image.shape).to(device)
      cHH = torch.from_numpy(np.stack((cH[0], cH[1], cH[2]), axis=-1)).reshape(*cover_image.shape).to(device)
      cVV = torch.from_numpy(np.stack((cV[0], cV[1], cV[2]), axis=-1)).reshape(*cover_image.shape).to(device)
      cDD = torch.from_numpy(np.stack((cD[0], cD[1], cD[2]), axis=-1)).reshape(*cover_image.shape).to(device)

      sAA = torch.from_numpy(np.stack((sA[0], sA[1], sA[2]), axis=-1)).reshape(*secret_image_1.shape).to(device)
      sHH = torch.from_numpy(np.stack((sH[0], sH[1], sH[2]), axis=-1)).reshape(*secret_image_1.shape).to(device)
      sVV = torch.from_numpy(np.stack((sV[0], sV[1], sV[2]), axis=-1)).reshape(*secret_image_1.shape).to(device)
      sDD = torch.from_numpy(np.stack((sD[0], sD[1], sD[2]), axis=-1)).reshape(*secret_image_1.shape).to(device)
 
      hA,rA= model(cAA,sAA,sAA,'full')
      hH,rH= model(cHH,sHH,sHH,'full')
      hV,rV= model(cVV,sVV,sVV,'full')
      hD,rD= model(cDD,sDD,sDD,'full')

      rA = rA.to(torch.device('cpu')).detach().to(torch.long)[0].permute(1,2,0).numpy()
      rH = rH.to(torch.device('cpu')).detach().to(torch.long)[0].permute(1,2,0).numpy()
      rV = rV.to(torch.device('cpu')).detach().to(torch.long)[0].permute(1,2,0).numpy()
      rD = rD.to(torch.device('cpu')).detach().to(torch.long)[0].permute(1,2,0).numpy()

      # rA= model(hH,hH,hH,'decoder')[0].permute(1,2,0).detach().numpy()
      # rH= model(hH,hH,hH,'decoder')[0].permute(1,2,0).detach().numpy()
      # rV= model(hV,hV,hV,'decoder')[0].permute(1,2,0).detach().numpy()
      # rD= model(hD,hD,hD,'decoder')[0].permute(1,2,0).detach().numpy()
      # rA = rA * 255
      # rA = rA.to(torch.device('cpu'))
      # rA = rA.detach().to(torch.long)
      # rA = rA[0].permute(1,2,0).numpy()
      # im = Image.fromarray(r.astype(np.uint8))
      # im.save("revealed.png")

      # print(hA)
      # print("latest")
      # print(rA)
      for ch in range(3):
        soeff[ch]= [(rA[:,:,ch], (rH[:,:,ch], rV[:,:,ch], rD[:,:,ch]))]
     
    # test = Image.fromarray(np.uint8(cAA))
    # test.save("test.png")
      reveal_image_1 = iwtransform(soeff,image)
      print(np.array(reveal_image_1))
      reveal_image_1.save("new.png")
    if mode =='decoder':
      
      reveal_image_1= model(cover_image,cover_image,cover_image,mode)
      
      # dot_graph = torchviz.make_dot(model(cover_image,cover_image,cover_image,mode))
      # dot_graph.render("decoder.dot")
    elif mode =='encoder':
      hidden_image= model(cover_image,secret_image_1,secret_image_1,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,secret_image_1,secret_image_1,mode))
      # dot_graph.render("encoder.dot")
    elif mode == "full":
      hidden_image,reveal_image_1= model(cover_image,secret_image_1,secret_image_1,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,secret_image_1,secret_image_1,mode))
      # dot_graph.render("full.dot")
      cover_image = cover_image * 255
      cover_image = cover_image.to(torch.device('cpu'))
      cover_image = cover_image.detach().to(torch.long)
      secret_image_1 = secret_image_1 * 255
      secret_image_1 = secret_image_1.to(torch.device('cpu'))
      secret_image_1 = secret_image_1.detach().to(torch.long)
    if mode =='encoder' or mode == 'full':
      hidden_image[hidden_image>1] = 1
      hidden_image = hidden_image * 255
      hidden_image = hidden_image.to(torch.device('cpu'))
      hidden_image = hidden_image.detach().to(torch.long)
      h = hidden_image[0].permute(1,2,0).numpy()
      im = Image.fromarray(h.astype(np.uint8))
      im.save("/content/drive/MyDrive/Colab Notebooks/stego_img.png")
      if mode == 'encoder': return h
    if mode =='decoder' or mode == 'full':
      reveal_image_1[reveal_image_1>1] = 1
      reveal_image_1 = reveal_image_1 * 255
      reveal_image_1 = reveal_image_1.to(torch.device('cpu'))
      reveal_image_1 = reveal_image_1.detach().to(torch.long)
      r = reveal_image_1[0].permute(1,2,0).numpy()
      im = Image.fromarray(r.astype(np.uint8))
      im.save("/content/drive/MyDrive/Colab Notebooks/revealed.png")
      if mode == 'decoder': return r
    return {
        'cover_image_grid':cover_image[0].permute(1,2,0).numpy(),
        'secret_image_1_grid':secret_image_1[0].permute(1,2,0).numpy(),
        'hidden_image_grid':h,
        'reveal_image_1_grid':r,
    }
grids = predict(model,test_data_loader,'full')

In [ ]:
grids = predict(loaded_model,test_data_loader,'full')

TypeError: ignored

In [ ]:
type(grids['cover_image_grid'])

In [ ]:
from torchvision.utils import save_image


In [ ]:
def plot(grids):
    plt.figure(figsize=(15,8))
    
    plt.subplot(241)
    plt.title('Cover Image')
    plt.imshow(grids['cover_image_grid'])

    plt.subplot(242)
    plt.title('Secret Image')
    plt.imshow(grids['secret_image_1_grid'])

    plt.subplot(245)
    plt.title('Hidden Image')
    plt.imshow(grids['hidden_image_grid'])
    
    plt.subplot(246)
    plt.title('Reveal Image')
    plt.imshow(grids['reveal_image_1_grid'])

    plt.savefig('/content/drive/MyDrive/Colab Notebooks/Plot.png')
    plt.show()
    

In [ ]:
'''
data = next(iter(test_data_loader))

cover_image = data['cover_image']
cover_image = cover_image.to(device)

secret_image_1 = data['secret_image']
secret_image_1 = secret_image_1.to(device)

model.eval()
    
hidden_image,reveal_image_1 = model(cover_image,secret_image_1,secret_image_1,'full')
'''

In [ ]:
#make_dot(hidden_image).render('hidden_image',format='png')

In [ ]:
#make_dot(reveal_image_1).render('reveal_image',format='png')

In [ ]:
'''
def plot_loss(loss_list):
    plt.plot(torch.tensor(loss_list, device = 'cpu'))
    plt.show()
'''

In [ ]:
#plot_loss(training_full_model_loss_list)

In [ ]:
#plot_loss(decoder_loss_list)

In [ ]:
plot(grids)

In [ ]:
'''
for i in range(5):
    grids = predict(loaded_model,test_data_loader)
    plot(grids)
'''

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt
import pywt
plt.rcParams['figure.figsize'] = [16, 16]
plt.rcParams.update({'font.size': 18})

A = imread('/content/drive/MyDrive/Colab Notebooks/stego_img.png')
print(type(A))
B = np.mean(A, -1); # Convert RGB to grayscale

n = 1
w = 'db1'
coeff = [[],[]]
arr = [[],[],[]]
coeff_slices = [[],[],[]]
for ch in range(3):
  coeffs = pywt.swt2(A[:,:,ch],wavelet=w,level=n)
  # data = np.zeros_like(A)
  # print(data.shape)
  # print(coeffs[0])
  # coeffs[1] = pywt.wavedec2(A[:,:,1],wavelet=w,level=n)
  # coeffs[2] = pywt.wavedec2(A[:,:,2],wavelet=w,level=n)

  # normalize each coefficient array
  
  coeff[0] = coeffs[0][0] / np.abs(coeffs[0][0]).max()
  for detail_level in range(n):
      coeff[detail_level + 1] = [d/np.abs(d).max() for d in coeffs[0][detail_level + 1]]

  arr[ch], coeff_slices[ch] = np.array(pywt.coeffs_to_array(coeff))

final_img = np.dstack([arr[0]*255, arr[1]*255, arr[2]*255]).astype(np.uint8)
print(final_img.shape)

# a = arr[coeff_slices[1]['dd']]
plt.imshow(final_img,cmap='gray',vmin=-0.25,vmax=0.75)
# plt.savefig('/content/drive/MyDrive/Colab Notebooks/Plotwave4.png')
plt.show()

In [ ]:

r_np = imread('/content/drive/MyDrive/Colab Notebooks/Plotwave2.png')
g_np = imread('/content/drive/MyDrive/Colab Notebooks/Plotwave3.png')
b_np = imread('/content/drive/MyDrive/Colab Notebooks/Plotwave4.png')


# Add the channels to the final image
final_img = np.dstack([b_np[:,:,0], g_np[:,:,2], r_np[:,:,2]]).astype(np.uint8)

print(final_img.shape)
plt.imshow(final_img, interpolation='nearest')
plt.show()

In [ ]:
import numpy as np
import math, time, sys
from PIL import Image
class Arnold:

    def __init__(self, a:int, b:int, rounds:int):
        # Parameters
        self.__a = a
        self.__b = b
        self.__rounds = rounds

    def mapping(self, s:np.shape):
        x, y = np.meshgrid(range(s[0]), range(s[0]), indexing="ij")
        xmap = (self.__a*self.__b*x + x + self.__a*y) % s[0]
        ymap = (self.__b*x + y) % s[0]
        return xmap, ymap

    def inverseMapping(self, s:np.shape):
        x, y = np.meshgrid(range(s[0]), range(s[0]), indexing="ij")
        xmap = (x - self.__a*y) % s[0]
        ymap = (-self.__b*x + self.__a*self.__b*y + y) % s[0]
        return xmap, ymap

    def applyTransformTo(self, image:np.ndarray):
        xm, ym = self.mapping(image.shape)
        img = image
        for r in range(self.__rounds):
            img = img[xm, ym]
        return img

    def applyInverseTransformTo(self, image:np.ndarray):
        xm, ym = self.inverseMapping(image.shape)
        img = image
        for r in range(self.__rounds):
          img = img[xm, ym]
        return img

In [ ]:

image_path = "/content/drive/MyDrive/Colab Notebooks/revealed.png"

# Arnold Transform Parameters
a = 6
b = 40
rounds = 33

# Open the images
lena = np.array(Image.open(image_path))

print(" ~~~~  * PARAMETERS *  ~~~~ ")
arnold = Arnold(a, b, rounds)
print("\ta:\t", a)
print("\tb:\t", b)
print("\trounds:\t", rounds)

print("\n ~~~~  *  RESULTS   *  ~~~~ ")

start_time = time.time()
scrambled = arnold.applyTransformTo(lena)
exec_time = time.time() - start_time
print("Transform  execution time: %.6f " % exec_time, "sec")
im = Image.fromarray(scrambled)
im.save("/content/drive/MyDrive/Colab Notebooks/scrambled111.tif", format="TIFF")

start_time = time.time()
reconstructed = arnold.applyInverseTransformTo(scrambled)
exec_time = time.time() - start_time
print("Inverse T. execution time: %.6f " % exec_time, "sec")
im = Image.fromarray(reconstructed)
im.save("/content/drive/MyDrive/Colab Notebooks/reconstructed1111.tif", format="TIFF")


In [ ]:
pip install opencv-python


In [ ]:
fimg1 = cv2.imread(frames1[0])
fimg2 = cv2.imread(frames2[0])

In [ ]:
dimension1=fimg1.shape[:2]
dimension1= tuple(reversed(dimension1))
dimension2=fimg2.shape[:2]
dimension2 = tuple(reversed(dimension2))

In [ ]:
dimension1

(600, 600)

In [ ]:
# dimension1 = framelist1[0].shape[:2]
# dimension1 = tuple(reversed(dimension1))
# dimension2 = framelist2[0].shape[:2]
# dimension2 = tuple(reversed(dimension2))

In [ ]:
fdiff = len(dirs2)-len(dirs1)
print("frames of secret - frames of cover = ", fdiff)

frames of secret - frames of cover =  -1509


In [ ]:
import random
def increaseFrames(framelist,diff):
  dlist=[]
  for i in range(0,diff):
    temp = random.randint(0, len(dirs1)-1)
    if temp not in dlist:
      dlist.append(temp)
    else : 
      while temp in dlist:
        temp = random.randint(0,len(dirs1)-1)
      dlist.append(temp)
  newlist=[]
  for i,frame in enumerate(framelist):
    if i not in dlist:
      newlist.append(frame)
    else:
      newlist.append(frame)
      newlist.append(frame)
  return newlist, dlist
      

In [ ]:

# if fdiff > 0:
#   modframelist1, modframes=increaseFrames(framelist1,fdiff)
#   framelist1 = modframelist1

if fdiff > 0:
  modframelist1, modframes = increaseFrames(frames1,fdiff)
  frames1 = modframelist1
  del modframelist1




In [ ]:
# stego_video=[]
# revealed_video = []
stego_dir= os.path.join(root_dir,'Stego-frames')

if not os.path.isdir(stego_dir):
        os.mkdir(stego_dir)
for i in range(len(frames1)):
  if len(frames2) > i:
    cover = Image.open(frames1[i])
    secret = Image.open(frames2[i])
    # cover = Image.fromarray(cover)
    # secret = Image.fromarray(secret)
    vgrids = predict(loaded_model,test_data_loader,'full')
    stego_frame = vgrids['hidden_image_grid']
   
    im = Image.fromarray(stego_frame.astype(np.uint8))
    im.save(os.path.join(stego_dir,f'0000{i}.png'))
    


    # revealed_video.append(vgrids['reveal_image_1_grid'])
  else:
    Image.open(frames1[i]).save(os.path.join(stego_dir,f'0000{i}.png'))

In [ ]:
stego_vid = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/steggy_vid.mp4',cv2.VideoWriter_fourcc(*'MP4V'), fps1, dimension1)

In [ ]:
revealed_vid = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/revealed_vid_.mp4',cv2.VideoWriter_fourcc(*'MP4V'), fps2, dimension1)

In [ ]:
dirsss=os.listdir(os.path.join(root_dir,'Stego-frames'))

In [ ]:
frames3 = [os.path.join(root_dir,'Stego-frames',item) for item in dirsss]

In [ ]:
for frame in frames3:
    vidframe = cv2.imread(frame)
    stego_vid.write(vidframe)

In [ ]:
stego_vid.release()
# revealed_vid.release()

In [ ]:
revealed_dir= os.path.join(root_dir,'revealed-frames')

if not os.path.isdir(revealed_dir):
        os.mkdir(revealed_dir)
for i in range(len(frames3)):
  cover = Image.open(frames3[i])
  
  # cover = Image.fromarray(cover)
  # secret = Image.fromarray(secret)
  revealed_frame = predict(loaded_model,test_data_loader,'decoder')
 
  
  im = Image.fromarray(revealed_frame.astype(np.uint8))
  im.save(os.path.join(revealed_dir,f'0000{i}.png'))
  




In [ ]:
dirs=os.listdir(os.path.join(root_dir,'revealed-frames'))


In [ ]:
print(dirs[:5])

In [ ]:
len(dirs)

In [ ]:
len(frames4[:len(frames2)])

In [ ]:
frames4 = [os.path.join(root_dir,'revealed-frames',item) for item in dirs]

In [ ]:
for frame in frames4[:len(frames2)]:
    vidframe = cv2.imread(frame)
    revealed_vid.write(vidframe)

In [ ]:
revealed_vid.release()

<h1>Residue Codes</h1>

In [ ]:
cover = cv2.cvtColor(framelist1[0], cv2.COLOR_BGR2RGB)
secret = cv2.cvtColor(framelist2[0], cv2.COLOR_BGR2RGB)
cover = Image.fromarray(cover)
secret = Image.fromarray(secret)


cover = Image.open(cover)
secret = Image.open(secret)


In [ ]:
vgrids = predict(loaded_model,test_data_loader,'full')

In [ ]:
plot(vgrids)

In [ ]:

plt.imshow(cover)
plt.show()

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
cv2_imshow(modframelist1[0])

In [ ]:
# stego_video=[]
# revealed_video = []
for i in range(len(framelist1)):
  if len(framelist2) > i:
    cover = cv2.cvtColor(framelist1[i], cv2.COLOR_BGR2RGB)
    secret = cv2.cvtColor(framelist2[i], cv2.COLOR_BGR2RGB)
    cover = Image.fromarray(cover)
    secret = Image.fromarray(secret)
    vgrids = predict(loaded_model,test_data_loader,'full')
    # stego_video.append(vgrids['hidden_image_grid'])
    # revealed_video.append(vgrids['reveal_image_1_grid'])
  else:
    # stego_video.append(framelist1[i])
    



In [ ]:
import random
def increaseFrames(framelist,diff):
  dlist=[]
  for i in range(0,diff):
    temp = random.randint(0,framecount1-1)
    if temp not in dlist:
      dlist.append(temp)
    else : 
      while temp in dlist:
        temp = random.randint(0,framecount1-1)
      dlist.append(temp)
  newlist=[]
  for i,frame in enumerate(framelist):
    if i not in dlist:
      newlist.append(frame)
    else:
      newlist.append(frame)
      newlist.append(frame)
  return newlist, dlist
      

In [ ]:
dimension1 = framelist1[0].shape[:2]
dimension1 = tuple(reversed(dimension1))
dimension2 = framelist2[0].shape[:2]
dimension2 = tuple(reversed(dimension2))

In [ ]:
# def format_timedelta(td):
#     """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
#     omitting microseconds and retaining milliseconds"""
#     result = str(td)
#     try:
#         result, ms = result.split(".")
#     except ValueError:
#         return (result + ".00").replace(":", "-")
#     ms = int(ms)
#     ms = round(ms / 1e4)
#     return f"{result}.{ms:02}".replace(":", "-")

In [ ]:
# def get_saving_frames_durations(cap, saving_fps):
#     """A function that returns the list of durations where to save the frames"""
#     s = []
#     # get the clip duration by dividing number of frames by the number of frames per second
#     clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
#     # use np.arange() to make floating-point steps
#     for i in np.arange(0, clip_duration, 1 / saving_fps):
#         s.append(i)
#     return s

In [ ]:
# def toFrames(video_file):
#     # filename, _ = os.path.splitext(video_file)
#     # filename += "-opencv"
#     # # make a folder by the name of the video file
#     # if not os.path.isdir(filename):
#     #     os.mkdir(filename)
#     # # read the video file    
#     cap = cv2.VideoCapture(video_file)
#     # get the FPS of the video
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     # # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
#     # saving_frames_per_second = fps
#     # # get the list of duration spots to save
#     # saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
#     # start the loop
#     count = 0
    
#     while True:
#         is_read, frame = cap.read()
#         if not is_read:
#             # break out of the loop if there are no frames to read
#             break
            
#         # # get the duration by dividing the frame count by the FPS
#         # frame_duration = count / fps
#         # try:
#         # #     # get the earliest duration to save
#         #     closest_duration = saving_frames_durations[0]
#         # except IndexError:
#         # #     # the list is empty, all duration frames were saved
#         #     break
#         # if frame_duration >= closest_duration:
#         # #     if closest duration is less than or equals the frame duration, 
#         # #     then save the frame
          
#           # frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
#         print(frame)
#             # drop the duration spot from the list, since this duration spot is already saved
#           # try:
#           #     saving_frames_durations.pop(0)
#           # except IndexError:
#           #     pass
#         # increment the frame count
#         # framelist.append(frame)
#         count += 1
#     print('total number of frames =', count)
#     # return framelist
